In [20]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import nltk
import plotly.express as px
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
sns.set()
data = pd.read_csv("spotify_songs.csv")
data.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  32833 non-null  object 
 1   track_name                32828 non-null  object 
 2   track_artist              32828 non-null  object 
 3   track_popularity          32833 non-null  int64  
 4   track_album_id            32833 non-null  object 
 5   track_album_name          32828 non-null  object 
 6   track_album_release_date  32833 non-null  object 
 7   playlist_name             32833 non-null  object 
 8   playlist_id               32833 non-null  object 
 9   playlist_genre            32833 non-null  object 
 10  playlist_subgenre         32833 non-null  object 
 11  danceability              32833 non-null  float64
 12  energy                    32833 non-null  float64
 13  key                       32833 non-null  int64  
 14  loudne

In [22]:
data.isnull().sum()

,0
track_id,0
track_name,5
track_artist,5
track_popularity,0
track_album_id,0
track_album_name,5
track_album_release_date,0
playlist_name,0
playlist_id,0
playlist_genre,0


In [23]:
data["artist"] = data.track_artist.astype(str).apply(lambda x: x.split(' ')[-1] if ' ' in x else x)
print(data.artist.unique())

['Sheeran' '5' 'Larsson' ... 'Wahlstrom' 'Sara' 'Zo']


In [24]:
data.artist

,artist
0,Sheeran
1,5
2,Larsson
3,Chainsmokers
4,Capaldi
...,...
32828,Simon
32829,Sara
32830,Starkillers
32831,Zo


In [25]:
df = data.drop(columns=['track_id','track_album_id', 'playlist_id','playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', ])
df.columns

Index(['track_name', 'track_artist', 'track_popularity', 'track_album_name',
       'track_album_release_date', 'playlist_name', 'playlist_genre',
       'duration_ms', 'artist'],
      dtype='object')

In [33]:
data['artist']=data['artist'].fillna('No artist Specified')
filtered_artists=pd.DataFrame()
filtered_artists=data['artist'].str.split(',',expand=True).stack()
filtered_artists=filtered_artists.to_frame()
filtered_artists.columns=['Artists']
artists=filtered_artists.groupby(['Artists']).size().reset_index(name='Total Content')
artists=artists[artists.Artists !='No Artists Specified']
artists=artists.sort_values(by=['Total Content'],ascending=False)
directorsTop5=artists.head()
directorsTop5=directorsTop5.sort_values(by=['Total Content'])
fig1=px.bar(directorsTop5,x='Total Content',y='Artists',title='Top 5 Artists on Spotify')
fig1.show()

In [34]:
data['artist'] = data['artist'].str.lower()

In [35]:
def recommend_song(name, playlist_genre):
    playlist_genre = playlist_genre.lower()
    word_tokenize(playlist_genre)
    stop_words = stopwords.words('english')
    lemm = WordNetLemmatizer()
    filtered  = {word for word in playlist_genre if not word in stop_words}
    filtered_set = set()
    for fs in filtered:
        filtered_set.add(lemm.lemmatize(fs))

    artist = data[data['artist']==name.lower()]
    artist = artist.set_index(np.arange(artist.shape[0]))
    list1 = []; list2 = []; cos = [];
    for i in range(artist.shape[0]):
        temp_token = word_tokenize(artist["artist"][i])
        temp_set = [word for word in temp_token if not word in stop_words]
        temp2_set = set()
        for s in temp_set:
            temp2_set.add(lemm.lemmatize(s))
        vector = temp2_set.intersection(filtered_set)
        cos.append(len(vector))
    artist['similarity']=cos
    artist = artist.sort_values(by='similarity', ascending=False)
    artist.drop_duplicates(subset='track_name', keep='first', inplace=True)
    artist.sort_values('track_popularity', ascending=False, inplace=True)
    artist.reset_index(inplace=True)
    return artist[["track_name", "track_popularity", "artist"]].head()

In [36]:
recommend_song('Mike', 'pop')

,track_name,track_popularity,artist
0,Instagram,81,mike
1,Boomshakalaka,74,mike
2,Boomshakalaka - Dimitri Vegas & Like Mike vs. ...,71,mike
3,Beast (All as One),71,mike
4,Selfish,69,mike
